In [ ]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

In [ ]:
trainDir = "../input/chest-xray-pneumonia/chest_xray/train"
testDir = "../input/chest-xray-pneumonia/chest_xray/test"
valDir = "../input/chest-xray-pneumonia/chest_xray/val"

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range =40, 
    width_shift_range=0.3, 
    height_shift_range=0.3,
    horizontal_flip=True, 
    
)
test_datagen = ImageDataGenerator(
    rescale = 1.0/255
    
)
val_datagen = ImageDataGenerator(
    rescale = 1.0/255
    
)

In [ ]:
BATCH_SIZE = 16
train_data = train_datagen.flow_from_directory(
    trainDir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    class_mode = "binary",
    batch_size = BATCH_SIZE
)
test_data = test_datagen.flow_from_directory(
    testDir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    class_mode = "binary",
    batch_size = BATCH_SIZE
)
val_data = val_datagen.flow_from_directory(
    valDir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    class_mode = "binary",
    batch_size = BATCH_SIZE
)

In [ ]:
mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False,
    weights='imagenet', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH,3), 
    pooling='avg'
)
mobilenet.trainable = False

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.keras import Model



inputs = Input(shape = (IMG_HEIGHT,IMG_WIDTH, 3))

pretrained_model = mobilenet(inputs, training=False)

dense_layer = Dense(1024, activation='relu')(pretrained_model)

outputs = Dense(1, activation='sigmoid')(dense_layer)

model = Model(inputs,outputs)

print(model.summary())

In [ ]:
EPOCHS = 5
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

model.fit(train_data,
         validation_data= val_data,
         batch_size = 16,
         epochs = EPOCHS,
         verbose =1)

In [ ]:
results = model.evaluate(test_data,verbose=1)
print(results[1]*100)